In [1]:
import numpy as np
import tensorflow as tf
import scipy.io
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn import manifold
import matplotlib as plt

In [2]:
fileidxJW11 = scipy.io.loadmat("XRMB/DATA/FILEIDX/fileidxJW11.mat")
fileidxJW13 = scipy.io.loadmat("XRMB/DATA/FILEIDX/fileidxJW13.mat")
fileidxJW24 = scipy.io.loadmat("XRMB/DATA/FILEIDX/fileidxJW24.mat")
fileidxJW30 = scipy.io.loadmat("XRMB/DATA/FILEIDX/fileidxJW30.mat")

JW11 = scipy.io.loadmat("XRMB/DATA/MAT/JW11[numfr1=7,numfr2=7].mat")
JW13 = scipy.io.loadmat("XRMB/DATA/MAT/JW13[numfr1=7,numfr2=7].mat")
JW24 = scipy.io.loadmat("XRMB/DATA/MAT/JW24[numfr1=7,numfr2=7].mat")
JW30 = scipy.io.loadmat("XRMB/DATA/MAT/JW30[numfr1=7,numfr2=7].mat")

In [7]:
print JW11.keys()

scaler = preprocessing.StandardScaler()
mfcc_features = scaler.fit_transform(preprocessing.normalize(np.transpose(JW11['MFCC'])))
articulatory_features = scaler.fit_transform(preprocessing.normalize(np.transpose(JW11['X']).astype(float)))
phone_labels = np.transpose(JW11['P'][0])

lb = preprocessing.LabelBinarizer() #one hot encoding of labels
lb.fit(phone_labels)
binarized_labels = lb.transform(phone_labels)

n_samples = mfcc_features.shape[0]
n_mfcc_features = mfcc_features.shape[1]
n_articulatory_features = articulatory_features.shape[1]

permutation = np.random.permutation(n_samples)
X1 = np.asarray([mfcc_features[i] for i in permutation])
X2 = np.asarray([articulatory_features[i] for i in permutation])
Y = np.asarray([binarized_labels[i] for i in permutation])
Phones = np.asarray([phone_labels[i] for i in permutation])

train, dev, test = 25948, 40948, 50948 #15948, 25948, 40948 #use 25948, 40948, 50948

X1_tr = X1[0:train, :]
X1_dev = X1[train:dev, :]
X1_test = X1[dev:test, :]

X2_tr = X2[0:train, :]

Y_tr = Y[0:train, :]
Y_dev = Y[train:dev, :]
Y_test = Y[dev:test, :]

Phones_tr = Phones[0:train]
Phones_dev = Phones[train:dev]
Phones_test = Phones[dev:test]

baseline_acoustic_tr = X1_tr[:, 118:157]
baseline_acoustic_dev = X1_dev[:, 118:157]
baseline_acoustic_test = X1_test[:, 118:157]
    
print X1_tr.shape
print X2_tr.shape
print Y_tr.shape
print baseline_acoustic_tr.shape

print X1_tr.shape
print X1_dev.shape
print X1_test.shape
print Phones_tr.shape

['bigindices', 'Valid_Files', 'indices', 'MFCC', 'Phones', 'bigP', '__header__', '__globals__', 'P', 'bigPhones', 'Words', 'frame_locs', 'Frames', 'X', '__version__', 'SpeakerID', 'Times']
(25948, 273)
(25948, 112)
(25948, 39)
(25948, 39)
(25948, 273)
(15000, 273)
(10000, 273)
(25948,)


In [21]:
#linear classifier
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01)) #generate a variable to store matrices, etc
    #anything to be updated is a variable, else a constant. placeholders not updated (such as data)

def model(X, w): #x placeholder for data, w is weight matrix, 
    return tf.matmul(X, w) #matmul is feedforward, 

X = tf.placeholder("float", [None, 273]) #none means unspecified number of instances, is a variable
Y = tf.placeholder("float", [None, 39])

w = init_weights([273, 39])

py_x = model(X, w) #sets node of computation graph

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y)) #cost function for NN
train_op = tf.train.GradientDescentOptimizer(0.001).minimize(cost) #train operation, define optimizer
predict_op = tf.argmax(py_x, 1) #bc one-hot encoded all labels, output label is one with most weight

sess = tf.Session() #could be interactive
init = tf.initialize_all_variables() #before evaluations, or specify which variables to be init
sess.run(init)

for i in range(100): #100 epochs
    for start, end in zip(range(0, len(X1_tr), 100), range(100, len(X1_tr), 100)): #batches of 100
        sess.run(train_op, feed_dict = {X: X1_tr[start:end], Y: Y_tr[start:end]})
        #include feed dictionary to provide actual values in placeholders used in the computation
    print i, np.mean(np.argmax(Y_dev, axis=1) == sess.run(predict_op, feed_dict = {X: X1_dev, Y: Y_dev}))
    
print np.mean(np.argmax(Y_test, axis=1) == sess.run(predict_op, feed_dict = {X: X1_test, Y: Y_test}))
#evaluate accuracy

0 0.254333333333
1 0.298666666667
2 0.317533333333
3 0.3332
4 0.347
5 0.361733333333
6 0.372666666667
7 0.383133333333
8 0.391533333333
9 0.399133333333
10 0.4054
11 0.412333333333
12 0.418066666667
13 0.4256
14 0.432266666667
15 0.4382
16 0.442533333333
17 0.446466666667
18 0.4506
19 0.454466666667
20 0.457266666667
21 0.459466666667
22 0.461666666667
23 0.464133333333
24 0.466933333333
25 0.469866666667
26 0.472533333333
27 0.475933333333
28 0.4778
29 0.4804
30 0.4826
31 0.485933333333
32 0.487533333333
33 0.489933333333
34 0.491
35 0.493533333333
36 0.495133333333
37 0.496866666667
38 0.498466666667
39 0.499866666667
40 0.501666666667
41 0.503266666667
42 0.505
43 0.506466666667
44 0.507466666667
45 0.508266666667
46 0.5104
47 0.512133333333
48 0.513
49 0.513533333333
50 0.5152
51 0.516333333333
52 0.517333333333
53 0.5186
54 0.520133333333
55 0.521466666667
56 0.522866666667
57 0.523933333333
58 0.525333333333
59 0.526266666667
60 0.527066666667
61 0.528133333333
62 0.528666666667


In [205]:
#autoencoder with logreg fine tuning with single hidden layer
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def lr_model(X, baseline, w_h, w_o):
    final = tf.concat(1, [X, baseline]) #concatenate baseline with learned representation
    h = tf.nn.sigmoid(tf.matmul(final, w_h)) #linear classifier on top
    return tf.matmul(h, w_o)

def encoder(X, enc1_w, enc1_b, enc2_w, enc2_b, enc3_w, enc3_b, enc4_w, enc4_b, p_keep_input, p_keep_hidden):
    X = tf.nn.dropout(X, p_keep_input) #give layer and probability to keep input
    h1 = tf.nn.relu(tf.matmul(X, enc1_w) + enc1_b)
    
    h1 = tf.nn.dropout(h1, p_keep_hidden)
    h2 = tf.nn.relu(tf.matmul(h1, enc2_w) + enc2_b)
    
    h2 = tf.nn.dropout(h2, p_keep_hidden)
    h3 = tf.nn.relu(tf.matmul(h2, enc3_w) + enc3_b)
    
    h3 = tf.nn.dropout(h3, p_keep_hidden)
    #3 layers with linear output layer
    return tf.matmul(h3, enc4_w) + enc4_b
    
def decoder_X1(h4, dec1_w, dec1_b, dec2_w, dec2_b, dec3_w, dec3_b, dec4_w, dec4_b, p_keep_hidden):
    h4 = tf.nn.dropout(h4, p_keep_hidden)
    h3 = tf.nn.relu(tf.matmul(h4, dec4_w) + dec4_b)
    
    h3 = tf.nn.dropout(h3, p_keep_hidden)
    h2 = tf.nn.relu(tf.matmul(h3, dec3_w) + dec3_b)
    
    h2 = tf.nn.dropout(h2, p_keep_hidden)
    h1 = tf.nn.relu(tf.matmul(h2, dec2_w) + dec2_b)
    
    h1 = tf.nn.dropout(h1, p_keep_hidden)
    return tf.matmul(h1, dec1_w) + dec1_b

def decoder_X2(h4, dec4_w, dec4_b, p_keep_hidden): #just a single layer for view2 reconstruction
    h4 = tf.nn.dropout(h4, p_keep_hidden)
    return tf.matmul(h4, dec4_w) + dec4_b

# def decoder_X2(h4, dec1_w, dec1_b, dec2_w, dec2_b, dec3_w, dec3_b, dec4_w, dec4_b, p_keep_hidden):
#     h4 = tf.nn.dropout(h4, p_keep_hidden)
#     h3 = tf.nn.sigmoid(tf.matmul(h4, dec4_w) + dec4_b)
    
#     h3 = tf.nn.dropout(h3, p_keep_hidden)
#     h2 = tf.nn.sigmoid(tf.matmul(h3, dec3_w) + dec3_b)
    
#     h2 = tf.nn.dropout(h2, p_keep_hidden)
#     h1 = tf.nn.sigmoid(tf.matmul(h2, dec2_w) + dec2_b)
    
#     h1 = tf.nn.dropout(h1, p_keep_hidden)
#     return tf.nn.sigmoid(tf.matmul(h1, dec1_w) + dec1_b)

X1 = tf.placeholder("float", [None, 273])
X2 = tf.placeholder("float", [None, 112])
Y = tf.placeholder("float", [None, 39])

#define widths of network
enc1_w = init_weights([273, 300])
enc2_w = init_weights([300, 150])
enc3_w = init_weights([150, 100])
enc4_w = init_weights([100, 50])

enc1_b = init_weights([1, 300])
enc2_b = init_weights([1, 150])
enc3_b = init_weights([1, 100])
enc4_b = init_weights([1, 50])

dec1_w_x1 = init_weights([300, 273])
dec2_w_x1 = init_weights([150, 300])
dec3_w_x1 = init_weights([100, 150])
dec4_w_x1 = init_weights([50, 100])

# dec1_w_x2 = init_weights([90, 112])
# dec2_w_x2 = init_weights([65, 90])
# dec3_w_x2 = init_weights([65, 65])
dec4_w_x2 = init_weights([50, 112])

dec1_b_x1 = init_weights([1, 273])
dec2_b_x1 = init_weights([1, 300])
dec3_b_x1 = init_weights([1, 150])
dec4_b_x1 = init_weights([1, 100])

# dec1_b_x2 = init_weights([1, 112])
# dec2_b_x2 = init_weights([1, 90])
# dec3_b_x2 = init_weights([1, 65])
dec4_b_x2 = init_weights([1, 112])

lr_w = init_weights([88, 50])
lr_w_o = init_weights([50, 39])
# lr_b = init_weights([1, 50])
# lr_b_o = init_weights([1, 39])

#dropouts are placeholders variables, but not updated during train

p_keep_input = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")

#
construction_x1 = encoder(X1, enc1_w, enc1_b, enc2_w, enc2_b, enc3_w, enc3_b, enc4_w, enc4_b, p_keep_input, p_keep_hidden)
reconstruction_x1 = decoder_X1(construction_x1, dec1_w_x1, dec1_b_x1, dec2_w_x1, dec2_b_x1, dec3_w_x1, dec3_b_x1, dec4_w_x1, dec4_b_x1, p_keep_hidden)
# reconstruction_x2 = decoder_X2(construction_x1, dec1_w_x2, dec1_b_x2, dec2_w_x2, dec2_b_x2, dec3_w_x2, dec3_b_x2, dec4_w_x2, dec4_b_x2, p_keep_hidden)
reconstruction_x2 = decoder_X2(construction_x1, dec4_w_x2, dec4_b_x2, p_keep_hidden)

#sum of reconstruction errors
ae_cost = tf.nn.l2_loss(reconstruction_x1 - X1) + tf.nn.l2_loss(reconstruction_x2 - X2)
ae_train_op = tf.train.RMSPropOptimizer(1e-4, 0.9).minimize(ae_cost) #momentum based optimizer
#ae_train_op = tf.train.AdamOptimizer(0.001).minimize(ae_cost)

#fine tuning
#all variables in chain updated by backprop
baseline = tf.placeholder("float", [None, 38])
py_x = lr_model(construction_x1, baseline, lr_w, lr_w_o)
lr_cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
lr_train_op = tf.train.AdamOptimizer(0.001).minimize(lr_cost) #root of chain of updates

predict_op = tf.argmax(py_x, 1)

sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

num_epochs = 500

#
print "autoencoder training: "
for i in range(num_epochs):
    for start, end in zip(range(0, len(X1_tr), 100), range(100, len(X1_tr), 100)):
        sess.run(ae_train_op, feed_dict = {X1: X1_tr[start:end], X2: X2_tr[start:end], Y: Y_tr[start:end], p_keep_input: 0.8, p_keep_hidden: 0.5})
    #print "Epoch ", i, ". Cost = ", sess.run(ae_cost, feed_dict = {X1: X1_tr, X2: X2_tr, Y: Y_tr, p_keep_input: 1.0, p_keep_hidden: 1.0})
    #print "" + str(i) + ", ",
print "done training autoencoder"
        
print "logistic regression training: "
for i in range(500):
    for start, end in zip(range(0, len(X1_tr), 100), range(100, len(X1_tr), 100)):
        sess.run(lr_train_op, feed_dict = {X1: X1_tr[start:end], Y: Y_tr[start:end], baseline: baseline_acoustic_tr[start:end], p_keep_input: 1.0, p_keep_hidden: 1.0})
    #print "Epoch ", i, ". cost = ", sess.run(lr_cost, feed_dict = {X1: X1_tr[start:end], Y: Y_tr[start:end], baseline: baseline_acoustic_tr[start:end], p_keep_input: 1.0, p_keep_hidden: 1.0})
    print i, np.mean(np.argmax(Y_dev, axis=1) == sess.run(predict_op, feed_dict = {X1: X1_dev, Y: Y_dev, baseline: baseline_acoustic_dev, p_keep_input: 1.0, p_keep_hidden: 1.0}))

 autoencoder training: 
done training autoencoder
logistic regression training: 
0 0.268126812681
1 0.343834383438
2 0.425442544254
3 0.46904690469
4 0.500050005001
5 0.5199519952
6 0.539453945395
7 0.552055205521
8 0.57095709571
9 0.579657965797
10 0.603860386039
11 0.614561456146
12 0.624862486249
13 0.640164016402
14 0.637063706371
15 0.641664166417
16 0.641864186419
17 0.663666366637
18 0.661666166617
19 0.670067006701
20 0.67596759676
21 0.686668666867
22 0.687068706871
23 0.687268726873
24 0.688768876888
25 0.686668666867
26 0.705370537054
27 0.699669966997
28 0.693069306931
29 0.694769476948
30 0.700670067007
31 0.69496949695
32 0.706170617062
33 0.703870387039
34 0.705870587059
35 0.708770877088
36 0.711771177118
37 0.704370437044
38 0.707170717072
39 0.708370837084
40 0.712871287129
41 0.701270127013
42 0.708470847085
43 0.707170717072
44 0.704170417042
45 0.716471647165
46 0.712571257126
47 0.719671967197
48 0.701270127013
49 0.70797079708
50 0.725472547255
51 0.715171517152


In [25]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def lr_model(X, baseline, w_o):
    final = tf.concat(1, [X, baseline])
    return tf.matmul(final, w_o)

def encoder(X, enc1_w, enc1_b, enc2_w, enc2_b, enc3_w, enc3_b, enc4_w, enc4_b, p_keep_input, p_keep_hidden):
    X = tf.nn.dropout(X, p_keep_input)
    h1 = tf.nn.relu(tf.matmul(X, enc1_w) + enc1_b)
    
    h1 = tf.nn.dropout(h1, p_keep_hidden)
    h2 = tf.nn.relu(tf.matmul(h1, enc2_w) + enc2_b)
    
    h2 = tf.nn.dropout(h2, p_keep_hidden)
    h3 = tf.nn.relu(tf.matmul(h2, enc3_w) + enc3_b)
    
    h3 = tf.nn.dropout(h3, p_keep_hidden)
    return tf.matmul(h3, enc4_w) + enc4_b
    
def decoder_X1(h4, dec1_w, dec1_b, dec2_w, dec2_b, dec3_w, dec3_b, dec4_w, dec4_b, p_keep_hidden):
    h4 = tf.nn.dropout(h4, p_keep_hidden)
    h3 = tf.nn.relu(tf.matmul(h4, dec4_w) + dec4_b)
    
    h3 = tf.nn.dropout(h3, p_keep_hidden)
    h2 = tf.nn.relu(tf.matmul(h3, dec3_w) + dec3_b)
    
    h2 = tf.nn.dropout(h2, p_keep_hidden)
    h1 = tf.nn.relu(tf.matmul(h2, dec2_w) + dec2_b)
    
    h1 = tf.nn.dropout(h1, p_keep_hidden)
    return tf.matmul(h1, dec1_w) + dec1_b

def decoder_X2(h4, dec4_w, dec4_b, p_keep_hidden):
    h4 = tf.nn.dropout(h4, p_keep_hidden)
    return tf.matmul(h4, dec4_w) + dec4_b

# def decoder_X2(h4, dec1_w, dec1_b, dec2_w, dec2_b, dec3_w, dec3_b, dec4_w, dec4_b, p_keep_hidden):
#     h4 = tf.nn.dropout(h4, p_keep_hidden)
#     h3 = tf.nn.sigmoid(tf.matmul(h4, dec4_w) + dec4_b)
    
#     h3 = tf.nn.dropout(h3, p_keep_hidden)
#     h2 = tf.nn.sigmoid(tf.matmul(h3, dec3_w) + dec3_b)
    
#     h2 = tf.nn.dropout(h2, p_keep_hidden)
#     h1 = tf.nn.sigmoid(tf.matmul(h2, dec2_w) + dec2_b)
    
#     h1 = tf.nn.dropout(h1, p_keep_hidden)
#     return tf.nn.sigmoid(tf.matmul(h1, dec1_w) + dec1_b)

X1 = tf.placeholder("float", [None, 273])
X2 = tf.placeholder("float", [None, 112])
Y = tf.placeholder("float", [None, 39])

enc1_w = init_weights([273, 500])
enc2_w = init_weights([500, 500])
enc3_w = init_weights([500, 500])
enc4_w = init_weights([500, 50])

enc1_b = init_weights([1, 500])
enc2_b = init_weights([1, 500])
enc3_b = init_weights([1, 500])
enc4_b = init_weights([1, 50])

dec1_w_x1 = init_weights([500, 273])
dec2_w_x1 = init_weights([500, 500])
dec3_w_x1 = init_weights([500, 500])
dec4_w_x1 = init_weights([50, 500])

# dec1_w_x2 = init_weights([90, 112])
# dec2_w_x2 = init_weights([65, 90])
# dec3_w_x2 = init_weights([65, 65])
dec4_w_x2 = init_weights([50, 112])

dec1_b_x1 = init_weights([1, 273])
dec2_b_x1 = init_weights([1, 500])
dec3_b_x1 = init_weights([1, 500])
dec4_b_x1 = init_weights([1, 500])

# dec1_b_x2 = init_weights([1, 112])
# dec2_b_x2 = init_weights([1, 90])
# dec3_b_x2 = init_weights([1, 65])
dec4_b_x2 = init_weights([1, 112])

lr_w_o = init_weights([88, 39])
# lr_b = init_weights([1, 50])
# lr_b_o = init_weights([1, 39])

p_keep_input = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")

construction_x1 = encoder(X1, enc1_w, enc1_b, enc2_w, enc2_b, enc3_w, enc3_b, enc4_w, enc4_b, p_keep_input, p_keep_hidden)
reconstruction_x1 = decoder_X1(construction_x1, dec1_w_x1, dec1_b_x1, dec2_w_x1, dec2_b_x1, dec3_w_x1, dec3_b_x1, dec4_w_x1, dec4_b_x1, p_keep_hidden)
# reconstruction_x2 = decoder_X2(construction_x1, dec1_w_x2, dec1_b_x2, dec2_w_x2, dec2_b_x2, dec3_w_x2, dec3_b_x2, dec4_w_x2, dec4_b_x2, p_keep_hidden)
reconstruction_x2 = decoder_X2(construction_x1, dec4_w_x2, dec4_b_x2, p_keep_hidden)

ae_cost = tf.nn.l2_loss(reconstruction_x1 - X1) + tf.nn.l2_loss(reconstruction_x2 - X2)
ae_train_op = tf.train.RMSPropOptimizer(1e-3, 0.99).minimize(ae_cost)
#ae_train_op = tf.train.AdagradOptimizer(1e-3).minimize(ae_cost)

baseline = tf.placeholder("float", [None, 38])
py_x = lr_model(construction_x1, baseline, lr_w_o)
lr_cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
lr_train_op = tf.train.AdamOptimizer(1e-3).minimize(lr_cost)

predict_op = tf.argmax(py_x, 1)

sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

num_epochs = 100

print "autoencoder training: "
for i in range(num_epochs):
    for start, end in zip(range(0, len(X1_tr), 100), range(100, len(X1_tr), 100)):
        sess.run(ae_train_op, feed_dict = {X1: X1_tr[start:end], X2: X2_tr[start:end], Y: Y_tr[start:end], p_keep_input: 0.8, p_keep_hidden: 0.5})
    #print "Epoch ", i, ". Cost = ", sess.run(ae_cost, feed_dict = {X1: X1_tr, X2: X2_tr, Y: Y_tr, p_keep_input: 1.0, p_keep_hidden: 1.0})
    print "" + str(i) + ", ",
print "done training autoencoder"
        
print "logistic regression training: "
for i in range(num_epochs):
    for start, end in zip(range(0, len(X1_tr), 100), range(100, len(X1_tr), 100)):
        sess.run(lr_train_op, feed_dict = {X1: X1_tr[start:end], Y: Y_tr[start:end], baseline: baseline_acoustic_tr[start:end], p_keep_input: 1.0, p_keep_hidden: 1.0})
    #print "Epoch ", i, ". cost = ", sess.run(lr_cost, feed_dict = {X1: X1_tr[start:end], Y: Y_tr[start:end], baseline: baseline_acoustic_tr[start:end], p_keep_input: 1.0, p_keep_hidden: 1.0})
    print i, np.mean(np.argmax(Y_dev, axis=1) == sess.run(predict_op, feed_dict = {X1: X1_dev, Y: Y_dev, baseline: baseline_acoustic_dev, p_keep_input: 1.0, p_keep_hidden: 1.0}))

autoencoder training: 
0,  1,  2,  3,  4,  5,  6,  7,  8,  9,  10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99,  done training autoencoder
logistic regression training: 
0 0.636863686369
1 0.705170517052
2 0.734673467347
3 0.749474947495
4 0.754375437544
5 0.755575557556
6 0.753175317532
7 0.759675967597
8 0.773777377738
9 0.765476547655
10 0.777077707771
11 0.783478347835
12 0.7899789979
13 0.79097909791
14 0.794479447945
15 0.791579157916
16 0.790579057906
17 0.79197919792
18 0.788478847885
19 0.800480048005
20 0.806780678068
21 0.805480548055
22 0.799579957996
23 0.790579057906
24 0.7946794

In [9]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def lr_model(X, baseline, w_o):
    final = tf.concat(1, [X, baseline])
    return tf.matmul(final, w_o)

def encoder(X, enc1_w, enc1_b, enc2_w, enc2_b, enc3_w, enc3_b, enc4_w, enc4_b, p_keep_input, p_keep_hidden):
    X = tf.nn.dropout(X, p_keep_input)
    h1 = tf.nn.relu(tf.matmul(X, enc1_w) + enc1_b)
    
    h1 = tf.nn.dropout(h1, p_keep_hidden)
    h2 = tf.nn.relu(tf.matmul(h1, enc2_w) + enc2_b)
    
    h2 = tf.nn.dropout(h2, p_keep_hidden)
    h3 = tf.nn.relu(tf.matmul(h2, enc3_w) + enc3_b)
    
    h3 = tf.nn.dropout(h3, p_keep_hidden)
    return tf.matmul(h3, enc4_w) + enc4_b
    
def decoder_X1(h4, dec1_w, dec1_b, dec2_w, dec2_b, dec3_w, dec3_b, dec4_w, dec4_b, p_keep_hidden):
    h4 = tf.nn.dropout(h4, p_keep_hidden)
    h3 = tf.nn.relu(tf.matmul(h4, dec4_w) + dec4_b)
    
    h3 = tf.nn.dropout(h3, p_keep_hidden)
    h2 = tf.nn.relu(tf.matmul(h3, dec3_w) + dec3_b)
    
    h2 = tf.nn.dropout(h2, p_keep_hidden)
    h1 = tf.nn.relu(tf.matmul(h2, dec2_w) + dec2_b)
    
    h1 = tf.nn.dropout(h1, p_keep_hidden)
    return tf.matmul(h1, dec1_w) + dec1_b

def decoder_X2(h4, dec3_w, dec3_b, dec4_w, dec4_b, p_keep_hidden):
    h4 = tf.nn.dropout(h4, p_keep_hidden)
    h3 = tf.matmul(h4, dec4_w) + dec4_b
    
    h3 = tf.nn.dropout(h3, p_keep_hidden)
    return tf.matmul(h3, dec3_w) + dec3_b

# def decoder_X2(h4, dec1_w, dec1_b, dec2_w, dec2_b, dec3_w, dec3_b, dec4_w, dec4_b, p_keep_hidden):
#     h4 = tf.nn.dropout(h4, p_keep_hidden)
#     h3 = tf.nn.sigmoid(tf.matmul(h4, dec4_w) + dec4_b)
    
#     h3 = tf.nn.dropout(h3, p_keep_hidden)
#     h2 = tf.nn.sigmoid(tf.matmul(h3, dec3_w) + dec3_b)
    
#     h2 = tf.nn.dropout(h2, p_keep_hidden)
#     h1 = tf.nn.sigmoid(tf.matmul(h2, dec2_w) + dec2_b)
    
#     h1 = tf.nn.dropout(h1, p_keep_hidden)
#     return tf.nn.sigmoid(tf.matmul(h1, dec1_w) + dec1_b)

X1 = tf.placeholder("float", [None, 273])
X2 = tf.placeholder("float", [None, 112])
Y = tf.placeholder("float", [None, 39])

enc1_w = init_weights([273, 1500])
enc2_w = init_weights([1500, 1500])
enc3_w = init_weights([1500, 1500])
enc4_w = init_weights([1500, 50])

enc1_b = init_weights([1, 1500])
enc2_b = init_weights([1, 1500])
enc3_b = init_weights([1, 1500])
enc4_b = init_weights([1, 50])

dec1_w_x1 = init_weights([1500, 273])
dec2_w_x1 = init_weights([1500, 1500])
dec3_w_x1 = init_weights([1500, 1500])
dec4_w_x1 = init_weights([50, 1500])

# dec1_w_x2 = init_weights([90, 112])
# dec2_w_x2 = init_weights([65, 90])
dec3_w_x2 = init_weights([75, 112])
dec4_w_x2 = init_weights([50, 75])

dec1_b_x1 = init_weights([1, 273])
dec2_b_x1 = init_weights([1, 1500])
dec3_b_x1 = init_weights([1, 1500])
dec4_b_x1 = init_weights([1, 1500])

# dec1_b_x2 = init_weights([1, 112])
# dec2_b_x2 = init_weights([1, 90])
dec3_b_x2 = init_weights([1, 112])
dec4_b_x2 = init_weights([1, 75])

lr_w_o = init_weights([89, 39])
# lr_b = init_weights([1, 50])
# lr_b_o = init_weights([1, 39])

p_keep_input = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")

construction_x1 = encoder(X1, enc1_w, enc1_b, enc2_w, enc2_b, enc3_w, enc3_b, enc4_w, enc4_b, p_keep_input, p_keep_hidden)
reconstruction_x1 = decoder_X1(construction_x1, dec1_w_x1, dec1_b_x1, dec2_w_x1, dec2_b_x1, dec3_w_x1, dec3_b_x1, dec4_w_x1, dec4_b_x1, p_keep_hidden)
# reconstruction_x2 = decoder_X2(construction_x1, dec1_w_x2, dec1_b_x2, dec2_w_x2, dec2_b_x2, dec3_w_x2, dec3_b_x2, dec4_w_x2, dec4_b_x2, p_keep_hidden)
reconstruction_x2 = decoder_X2(construction_x1, dec3_w_x2, dec3_b_x2, dec4_w_x2, dec4_b_x2, p_keep_hidden)

ae_cost = tf.nn.l2_loss(reconstruction_x1 - X1) + tf.nn.l2_loss(reconstruction_x2 - X2)
ae_train_op = tf.train.RMSPropOptimizer(1e-3, 0.99).minimize(ae_cost)
#ae_train_op = tf.train.AdagradOptimizer(1e-3).minimize(ae_cost)

baseline = tf.placeholder("float", [None, 39])
py_x = lr_model(construction_x1, baseline, lr_w_o)
lr_cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
lr_train_op = tf.train.AdamOptimizer(1e-3).minimize(lr_cost)

predict_op = tf.argmax(py_x, 1)

sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

num_epochs = 100

print "autoencoder training: "
for i in range(num_epochs):
    for start, end in zip(range(0, len(X1_tr), 100), range(100, len(X1_tr), 100)):
        sess.run(ae_train_op, feed_dict = {X1: X1_tr[start:end], X2: X2_tr[start:end], Y: Y_tr[start:end], p_keep_input: 0.8, p_keep_hidden: 0.5})
    #print "Epoch ", i, ". Cost = ", sess.run(ae_cost, feed_dict = {X1: X1_tr, X2: X2_tr, Y: Y_tr, p_keep_input: 1.0, p_keep_hidden: 1.0})
    print "" + str(i) + ", ",
print "done training autoencoder"
        
print "logistic regression training: "
for i in range(num_epochs):
    for start, end in zip(range(0, len(X1_tr), 100), range(100, len(X1_tr), 100)):
        sess.run(lr_train_op, feed_dict = {X1: X1_tr[start:end], Y: Y_tr[start:end], baseline: baseline_acoustic_tr[start:end], p_keep_input: 1.0, p_keep_hidden: 1.0})
    #print "Epoch ", i, ". cost = ", sess.run(lr_cost, feed_dict = {X1: X1_tr[start:end], Y: Y_tr[start:end], baseline: baseline_acoustic_tr[start:end], p_keep_input: 1.0, p_keep_hidden: 1.0})
    print i, np.mean(np.argmax(Y_dev, axis=1) == sess.run(predict_op, feed_dict = {X1: X1_dev, Y: Y_dev, baseline: baseline_acoustic_dev, p_keep_input: 1.0, p_keep_hidden: 1.0}))

 autoencoder training: 
0,  1,  2,  3,  4,  5,  6,  7,  8,  9,  10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99,  done training autoencoder
logistic regression training: 
0 0.693466666667
1 0.753733333333
2 0.7756
3 0.801733333333
4 0.8012
5 0.8092
6 0.812333333333
7 0.817733333333
8 0.824
9 0.830066666667
10 0.831133333333
11 0.835866666667
12 0.8336
13 0.8264
14 0.831
15 0.827
16 0.827933333333
17 0.833866666667
18 0.840066666667
19 0.839666666667
20 0.8438
21 0.838066666667
22 0.848466666667
23 0.841333333333
24 0.841533333333
25 0.837533333333
26 0.834933333333
27 0.8434
28 0.845066666667


In [10]:
def encoder(X, enc1_w, enc1_b, enc2_w, enc2_b, enc3_w, enc3_b, enc4_w, enc4_b, p_keep_input, p_keep_hidden):
    X = tf.nn.dropout(X, p_keep_input)
    h1 = tf.nn.relu(tf.matmul(X, enc1_w) + enc1_b)
    
    h1 = tf.nn.dropout(h1, p_keep_hidden)
    h2 = tf.nn.relu(tf.matmul(h1, enc2_w) + enc2_b)
    
    h2 = tf.nn.dropout(h2, p_keep_hidden)
    h3 = tf.nn.relu(tf.matmul(h2, enc3_w) + enc3_b)
    
    h3 = tf.nn.dropout(h3, p_keep_hidden)
    return tf.matmul(h3, enc4_w) + enc4_b

ae1_w = tf.constant(sess.run(enc1_w)) #now ae1 weights are fixed, used for rbm layerwise training
ae2_w = tf.constant(sess.run(enc2_w)) 
ae3_w = tf.constant(sess.run(enc3_w))
ae4_w = tf.constant(sess.run(enc4_w))

ae1_b = tf.constant(sess.run(enc1_b))
ae2_b = tf.constant(sess.run(enc2_b))
ae3_b = tf.constant(sess.run(enc3_b))
ae4_b = tf.constant(sess.run(enc4_b))

In [16]:
X1 = tf.placeholder("float", [None, 273])
baseline = tf.placeholder("float", [None, 39])

p_keep_input = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")

shared_rep = encoder(X1, ae1_w, ae1_b, ae2_w, ae2_b, ae3_w, ae3_b, ae4_w, ae4_b, p_keep_input, p_keep_hidden)
stacked = tf.concat(1, [shared_rep, baseline])

transformed_train = sess.run(stacked, feed_dict = {X1: X1_tr, baseline: baseline_acoustic_tr, p_keep_input: 1.0, p_keep_hidden: 1.0})
transformed_dev = sess.run(stacked, feed_dict = {X1: X1_dev, baseline: baseline_acoustic_dev, p_keep_input: 1.0, p_keep_hidden: 1.0})
transformed_test = sess.run(stacked, feed_dict = {X1: X1_test, baseline: baseline_acoustic_test, p_keep_input: 1.0, p_keep_hidden: 1.0})

In [17]:
scipy.io.savemat('splitae_projected_data.mat', {'dataTr': transformed_train, 'PhonesTr': Phones_tr, 'dataDev': transformed_dev, 'PhonesDev': Phones_dev, 'dataTest': transformed_test, 'PhonesTest': Phones_test})

In [15]:
lr_fixed_w_train = tf.constant(sess.run(lr_w_o))

In [19]:
def lr_model(X, baseline, w_o):
    final = tf.concat(1, [X, baseline])
    return tf.matmul(final, w_o)

X1 = tf.placeholder("float", [None, 273])
Y = tf.placeholder("float", [None, 39])
baseline = tf.placeholder("float", [None, 39])

lr_w_o = init_weights([89, 39])

p_keep_input = tf.placeholder("float")
p_keep_output = tf.placeholder("float")

shared = encoder(X1, ae1_w, ae1_b, ae2_w, ae2_b, ae3_w, ae3_b, ae4_w, ae4_b, p_keep_input, p_keep_hidden)
py_x = lr_model(shared, baseline, lr_w_o)
lr_cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
lr_train_op = tf.train.AdamOptimizer(1e-3).minimize(lr_cost)

predict_op = tf.argmax(py_x, 1)

init = tf.initialize_all_variables()
sess.run(init)

num_epochs = 100

for i in range(num_epochs):
    for start, end in zip(range(0, len(X1_tr), 100), range(100, len(X1_tr), 100)):
        sess.run(lr_train_op, feed_dict = {X1: X1_tr[start:end], Y: Y_tr[start:end], baseline: baseline_acoustic_tr[start:end], p_keep_input: 1.0, p_keep_hidden: 1.0})
    #print "Epoch ", i, ". cost = ", sess.run(lr_cost, feed_dict = {X1: X1_tr[start:end], Y: Y_tr[start:end], baseline: baseline_acoustic_tr[start:end], p_keep_input: 1.0, p_keep_hidden: 1.0})
    print i, np.mean(np.argmax(Y_dev, axis=1) == sess.run(predict_op, feed_dict = {X1: X1_dev, Y: Y_dev, baseline: baseline_acoustic_dev, p_keep_input: 1.0, p_keep_hidden: 1.0}))
    
print 'LR on test data'
print np.mean(np.argmax(Y_test, axis=1) == sess.run(predict_op, feed_dict = {X1: X1_test, Y: Y_test, baseline: baseline_acoustic_test, p_keep_input: 1.0, p_keep_hidden: 1.0}))

0 0.851466666667
1 0.8538
2 0.854533333333
3 0.854466666667
4 0.854066666667
5 0.853866666667
6 0.854266666667
7 0.854266666667
8 0.854066666667
9 0.853133333333
10 0.853266666667
11 0.852866666667
12 0.852866666667
13 0.852866666667
14 0.852866666667
15 0.8528
16 0.852466666667
17 0.853
18 0.8528
19 0.852733333333
20 0.852533333333
21 0.8526
22 0.852733333333
23 0.852666666667
24 0.852533333333
25 0.852533333333
26 0.8524
27 0.852333333333
28 0.852066666667
29 0.852133333333
30 0.8522
31 0.852
32 0.852133333333
33 0.851866666667
34 0.851933333333
35 0.851466666667
36 0.8518
37 0.850333333333
38 0.849133333333
39 0.850533333333
40 0.852133333333
41 0.851666666667
42 0.851133333333
43 0.851666666667
44 0.8514
45 0.851666666667
46 0.851733333333
47 0.8518
48 0.8518
49 0.8516
50 0.851533333333
51 0.851733333333
52 0.851933333333
53 0.851933333333
54 0.852066666667
55 0.852266666667
56 0.8522
57 0.8522
58 0.852066666667
59 0.852066666667
60 0.852133333333
61 0.852
62 0.8518
63 0.8517333333